<h1>LLM User flow test</h1>
Test and finalise user flow before applying changes

The current questioning for prerequisites feels a bit lengthy, without achieving the depth i envisioned ._.

General flow is: define goals -> get prereq info -> generate phases -> frontend should allow user to add/edit/delete phases, user can also comment on phases for llm to refine -> generate dailies based on phases -> same interaction

In [26]:
from pydantic import BaseModel, Field
from typing import Union, List, Literal, Any, Optional
from datetime import date
import uuid
import json
from google import genai
from google.genai import types

# this should match backend/schemas
# use goalrequest to send to fastapi
# dastapi to return apiresponse
# base classes are what should be stored in db (may not need id)
# read classes are is what is stored in db (includes id etc.)

class GoalRequest(BaseModel):
    user_input: str
    session_id: Union[str, None]

class APIResponse(BaseModel):
    session_id: str
    data: Any # 'Any' will be replaced by specific phase-related models

class FollowUp(BaseModel):
    status: Literal['follow_up_required'] = 'follow_up_required'
    question_to_user: str = Field(description="A single, specific, question to the user.")

class CurrentState(BaseModel):
    """Details about the user's starting point and gaps."""
    skill_level: str = Field(description="The user's current skill level related to the goal.")
    related_experience: str = Field(description="Relevant past experience or projects.")
    resources_available: str = Field(description="Current readily available resources (tools, software, people).")
    user_gap_assessment: List[str] = Field(description="A list of problems or missing skills the user has identified.")
    possible_gap_assessment: List[str] = Field(description="A list of problems or missing skills identified by the LLM (optional, for planning assistance).")

class FixedResources(BaseModel):
    """Non-negotiable resource constraints."""
    time_commitment_per_week_hours: float = Field(description="The number of hours the user can reliably commit per week.")
    budget: float = Field(description="The monetary budget available for the goal (in the user's local currency, e.g., 'USD').")
    required_equipment: str = Field(description="Specific equipment or materials needed to start or complete the goal.")
    support_system: str = Field(description="People or groups available for emotional or practical support.")

class Constraints(BaseModel):
    """Scheduling and external limitations."""
    blocked_time_blocks: List[str] = Field(description="Specific recurring time blocks (e.g., 'Mondays 9am-5pm') when the user is unavailable.")
    available_time_blocks: List[str] = Field(description="Specific recurring time blocks (e.g., 'Tuesday 7pm-9pm') when the user is free to work on the goal.")
    dependencies: List[str] = Field(description="A list of external requirements that must be met before the goal can progress (e.g., 'Wait for equipment delivery').")

class GoalPrerequisites(BaseModel):
    """The complete structure for all prerequisites."""
    status: Literal['prerequisites_extracted'] = 'prerequisites_extracted'
    current_state: CurrentState
    fixed_resources: FixedResources
    constraints: Constraints

class DefinitionsBase(BaseModel):
    title: str = Field(description="The specific goal to achieve")
    metric: str = Field(description="The objective, quantifiable metric.")
    purpose: str = Field(description="The underlying motivation or 'why'.")
    deadline: date = Field(description="The date the goal must be completed by, formatted as YYYY-MM-DD.")
    
class DefinitionsCreate(DefinitionsBase):
    status: Literal['definitions_extracted'] = 'definitions_extracted'

class Definitions(DefinitionsCreate):
    id: int
    prerequisites: GoalPrerequisites # Includes the data expected from Phase 2
    
    class Config:
        from_attributes = True 

class PhaseBase(BaseModel):
    title: str = Field(
        description="A descriptive title for the phase (e.g., 'Foundation Building', 'Skill Consolidation')."
    )
    description: str = Field(
        description="A detailed description of the phase's measurable target and objective (e.g., 'Run 10km under 60 minutes' or 'Complete chapters 1-5 and pass the unit test')."
    )
    start_date: date = Field(
        description="The estimated starting date for this phase, formatted as YYYY-MM-DD."
    )
    end_date: date = Field(
        description="The estimated completion date for this phase, formatted as YYYY-MM-DD. This date marks the achievement of the phase's target."
    )

class PhaseCreate(PhaseBase):
    pass # Used for POST request body

class PhaseGeneration(BaseModel):
    """Container for the variable list of generated phases."""
    status: Literal['phases_generated'] = 'phases_generated'
    phases: List[PhaseCreate] = Field(
        description="A list of 3 to 5 broad, sequential phases required to achieve the user's goal."
    )

class PhaseRead(PhaseBase):
    id: int
    goal_id: int
    is_completed: bool

    class Config:
        from_attributes = True

class DailyBase(BaseModel):
    task_description: str
    estimated_time_minutes: int

class DailyCreate(DailyBase):
    pass # Used for POST request body

class DailyRead(DailyBase):
    id: int
    phase_id: int
    is_completed: bool

    class Config:
        from_attributes = True

C:\Users\SK\AppData\Local\Temp\ipykernel_7808\2362590581.py:64: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  class Definitions(DefinitionsCreate):
C:\Users\SK\AppData\Local\Temp\ipykernel_7808\2362590581.py:95: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  class PhaseRead(PhaseBase):
C:\Users\SK\AppData\Local\Temp\ipykernel_7808\2362590581.py:110: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  class DailyRead(DailyBase):


In [ ]:
GOOGLE_API_KEY=""

GOAL_DEFINITION_INSTRUCTION = """
You are an expert Goal Planning Assistant designed to extract specific goal details from a user's input.
Your primary function is to conduct a **multiple-turn conversation** to fully define the user's goal by filling all required fields: title, metric, purpose, and deadline.
The date is currently {current_date_str}
Your response MUST be a JSON object matching one of the two Pydantic schemas below. **You should anticipate using the FollowUp schema in most turns.:

1. **FollowUp Schema (If information is missing):**
{followUp}

2. **DefinitionsCreate Schema (If all information is gathered):**
{definitionsCreate}

You must choose the appropriate schema based on whether all fields (title, metric, purpose, deadline) are complete. 
If any field is missing, you MUST ask ONLY ONE clarifying question using the FollowUp schema.

You must adhere to the following rules:
1.  **Extraction:** You must attempt to extract any available information for all fields from the current user input.
2.  **Strict Follow-Up Rule:** If **title**, **metric**, OR **purpose** are missing or unclear, you **MUST** use the `FollowUp` schema to ask **ONLY ONE** clarifying question per turn.
    * **Priority 1:** Clarify `title` if missing.
    * **Priority 2:** Clarify `purpose` if missing.
    * **Priority 3: Enforce Measurable Metric.** If the extracted `metric` is subjective or vague (e.g., "fluent," "performance ready," "feel great"), you **MUST** ask a clarifying question to get an **objective, quantifiable measure** (e.g., "Is there a test you could take to know that you 'fluent'? What score would you like to obtain on that test?" or "What score or number determines 'success' for your metric?").
    * **Exception:** If the metric refers to an existing external judgment (e.g., "teacher's approval"), you may accept it but should ask for the underlying, objective criteria if possible.
3.  **Deadline Handling:**
    * **NEVER** ask a clarifying question about the deadline field.
    * If no deadline is provided, infer the deadline (e.g., "by the end of the year") or estimate using the task's complexity (e.g. 'landing a job' -> estimate 1 year).
    * deadline should be in an explicit YYYY-MM-DD format.
4. **Metric Handling:**
    * Metric has to be quantifiable (e.g. 'land a job', '50%', 'ten thousand dollars in revenue', '3 runs with no mistakes')
    * Metric cannot be user subjective (e.g. 'performance ready', 'fluent'), but accept it if the user can get external approval (e.g. 'performance ready according to my teacher').
5.  **Output Format:** You **ONLY** use the `DefinitionsCreate` schema when **title**, **metric**, AND **purpose** are all definitively extracted and represented as non-empty strings. The state of the `deadline` field is irrelevant to this final decision.

---
DEMONSTRATION EXAMPLES START
---

Example 1: Demonstrating Follow-Up (Metric)

USER: I want to save money.
MODEL:
{{
  "status": "follow_up_required",
  "question_to_user": "That's an important goal to have! What are you saving up for?"
}}

USER: I am saving up for a downpayment on a new huse.
MODEL:
{{
  "status": "follow_up_required",
  "question_to_user": "Great! ow much would the estimated downpayment be?"
}}

USER: About $200000.
MODEL:
{{
  "status": "definitions_extracted",
  "title": "Saving money",
  "metric": "Accumulate $200000",
  "purpose": "Purcahse a new home",
  "deadline": 2028-01-01
}}
---

Example 2: Demonstrating Metric Measurability

USER: i want to learn to code so i can be good at python by this year end.
MODEL:
{{
  "status": "follow_up_required",
  "question_to_user": "Let's make that measurable. What defines being 'good at Python'? (e.g. 'complete a full-stack project', 'solve 5 leetcode hard questions')"
}}

USER: I want to finish a data analysis project.
MODEL:
{{
  "status": "definitions_extracted",
  "title": "Learn to code",
  "metric": "Finish a data analysis project",
  "purpose": "Be good at python",
  "deadline": 2025-12-31
}}

---
DEMONSTRATION EXAMPLES END
---
"""

In [ ]:
client = genai.Client(api_key=GOOGLE_API_KEY)
chat_sessions = {}

def goal_definition(usr_reqeust = GoalRequest):
    ResponseSchema = Union[DefinitionsCreate, FollowUp]
    if usr_reqeust.session_id:
        session_id = usr_reqeust.session_id
        chat = chat_sessions[session_id]
    else:
        session_id = str(uuid.uuid4())
        chat = client.chats.create(
            model='gemini-2.5-flash-lite',
            config={
                "system_instruction": GOAL_DEFINITION_INSTRUCTION.format(
                    current_date_str=date.today().strftime('%Y-%m-%d'),
                    followUp=FollowUp.model_json_schema(),
                    definitionsCreate=DefinitionsCreate.model_json_schema()
                ),
                "response_mime_type": "application/json",
                "response_schema": ResponseSchema,
            },
        )
        chat_sessions[session_id] = chat

    response = chat.send_message(usr_reqeust.user_input)
    results = response.parsed

    if isinstance(results, DefinitionsCreate):
        return APIResponse(session_id=session_id, data=results)
    elif isinstance(results, FollowUp):
        return APIResponse(session_id=session_id, data=results)

usr_input = input("I want to: ")
usr_reqeust = GoalRequest(user_input=usr_input, session_id=None)
apiresponse = goal_definition(usr_reqeust)
print(apiresponse)
while apiresponse.data.status == "follow_up_required":
    usr_input = input(apiresponse.data.question_to_user)
    usr_reqeust = GoalRequest(user_input=usr_input, session_id=apiresponse.session_id)
    apiresponse = goal_definition(usr_reqeust)
    print(apiresponse)
#We now have the definitions extracted. We will also confirm the deadline using a calander picker in the frontend. (notice how the prompt does not require deadline)

#session_id='9ca24eab-5030-4d77-8588-9a0d7573dcbe' data=DefinitionsCreate(title='Learn the guitar', metric="Passing the guitar club's interview", purpose='Joining the NUS guitar club', deadline=datetime.date(2026, 11, 14), status='definitions_extracted')

session_id='9ca24eab-5030-4d77-8588-9a0d7573dcbe' data=FollowUp(status='follow_up_required', question_to_user="That's a fantastic goal! To make sure I understand, what do you want to achieve by learning the guitar? For example, do you want to play a specific song, join a band, or just play for enjoyment?")
session_id='9ca24eab-5030-4d77-8588-9a0d7573dcbe' data=FollowUp(status='follow_up_required', question_to_user="That's a great purpose! To make 'learning the guitar' a concrete goal, what specific skill or achievement would signify you've learned it? For example, 'play 5 songs' or 'complete a beginner's course'?")
session_id='9ca24eab-5030-4d77-8588-9a0d7573dcbe' data=DefinitionsCreate(title='Learn the guitar', metric="Passing the guitar club's interview", purpose='Joining the NUS guitar club', deadline=datetime.date(2026, 11, 14), status='definitions_extracted')


In [20]:
GOAL_PREREQUISITE_INSTRUCTION = """
You are an expert Goal Prerequisite Analyst. Your task is to conduct a multi-turn conversation to systematically extract the user's starting point, available resources, and constraints necessary to achieve their defined goal. You must actively research common prerequisites for the user's goal and inquire about them.

Your goal is to fill all 11 fields across the three nested schemas in the final GoalPrerequisites object.
Your response MUST be a JSON object matching one of the two Pydantic schemas below. **You should anticipate using the FollowUp schema in most turns.:

1. **FollowUp Schema (If information is missing):**
{followUp}
2. **DefinitionsCreate Schema (If all information is gathered):**
{goalPrerequisites}

### Strict Rules and Follow-Up Logic:
1.  **Context:** The conversation starts with the goal context provided by the application. You must acknowledge the goal and begin the extraction with the highest priority question (Skill Level).
2.  **Extraction:** You must attempt to extract any available information for *any* field from the current user input.
3.  **Single Question Rule:** If any of the 11 fields are missing or unclear, you **MUST** use the `FollowUp` schema to ask **ONLY ONE** clarifying question per turn.
4.  **Research-Driven Questioning:** You **MUST** research likely necessary resources based on the goal (e.g., "Given your goal, do you have [researched item]?"). This helps uncover missing prerequisites the user hasn't considered.
5.  **Priority Order:** Ask questions in the following priority:
    a. `skill_level` (current ability related to the goal), quantified where possible.
    b. `time_commitment_per_week_hours` (reliable weekly work hours).
    c. `resources_available` (what the user has now).
    d. `required_equipment` (physical materials needed - research if needed).
    e. `budget` (monetary limit).
    f. `support_system` (people/groups - research likely types if needed).
    g. All other fields in the order they appear in the schemas.
6.  **LLM Gap Assessment:** The `possible_gap_assessment` list is **optional**. Do not ask the user for this information. Only fill it in when all other fields are complete and you are preparing the final `GoalPrerequisites` output, by listing potential prerequisites that **you researched but the user indicated they do not have**.
7.  **Final Completion Rule:** If all 11 fields have been successfully extracted (with `possible_gap_assessment` optionally filled), use the `GoalPrerequisites` schema.

The date is currently {current_date_str}
"""

In [22]:
client = genai.Client(api_key=GOOGLE_API_KEY)
chat_sessions = {}

user_goal=DefinitionsCreate(title='Learn the guitar', metric="Passing the guitar club's interview", purpose='Joining the NUS guitar club', deadline=date(2026, 11, 14), status='definitions_extracted')
established_goal = f"""
Here is my goal:
goal: {user_goal.title}
purpose: {user_goal.purpose}
metric: {user_goal.metric}
deadline: {user_goal.deadline.strftime('%Y-%m-%d')}
What prerequisites do you need?
"""

def goal_prereq(usr_reqeust = GoalRequest):
    ResponseSchema = Union[GoalPrerequisites, FollowUp]
    if usr_reqeust.session_id:
        session_id = usr_reqeust.session_id
        chat = chat_sessions[session_id]
    else:
        session_id = str(uuid.uuid4())
        chat = client.chats.create(
            model='gemini-2.5-flash-lite',
            config={
                "system_instruction": GOAL_PREREQUISITE_INSTRUCTION.format(
                    current_date_str=date.today().strftime('%Y-%m-%d'),
                    followUp=FollowUp.model_json_schema(),
                    goalPrerequisites=GoalPrerequisites.model_json_schema()
                ),
                "response_mime_type": "application/json",
                "response_schema": ResponseSchema,
            },
        )
        chat_sessions[session_id] = chat

    response = chat.send_message(usr_reqeust.user_input)
    results = response.parsed

    if results == None:
        candidates_list = response.candidates
        first_candidate = candidates_list[0]
        content_object = first_candidate.content
        parts_list = content_object.parts
        json_string = parts_list[0].text
        if json_string.startswith('```json'):
            json_string = json_string[len('```json'):]
        if json_string.endswith('```'):
            json_string = json_string[:-len('```')]
        parsed_dict = json.loads(json_string)
        try:
            results = FollowUp.model_validate(parsed_dict)
        except:
            results = GoalPrerequisites.model_validate(parsed_dict)

    if isinstance(results, GoalPrerequisites):
        return APIResponse(session_id=session_id, data=results)
    elif isinstance(results, FollowUp):
        return APIResponse(session_id=session_id, data=results)

apiresponse = goal_prereq(GoalRequest(user_input=established_goal, session_id=None))
print(apiresponse)
while apiresponse.data.status == "follow_up_required":
    usr_input = input(apiresponse.data.question_to_user)
    print(usr_input)
    usr_reqeust = GoalRequest(user_input=usr_input, session_id=apiresponse.session_id)
    apiresponse = goal_prereq(usr_reqeust)
    print(apiresponse)

session_id='f350dbbd-ab99-46dd-86a1-65d54ecdfcd1' data=FollowUp(status='follow_up_required', question_to_user='What is your current skill level with the guitar? For example, are you a complete beginner, intermediate, or advanced player?')
i have some experience
session_id='f350dbbd-ab99-46dd-86a1-65d54ecdfcd1' data=FollowUp(status='follow_up_required', question_to_user='How many hours per week can you realistically dedicate to practicing the guitar?')
30mins on weekends
session_id='f350dbbd-ab99-46dd-86a1-65d54ecdfcd1' data=FollowUp(status='follow_up_required', question_to_user='What resources do you currently have available for learning guitar, such as a guitar, amplifier, or online learning subscriptions?')
i have a guitar
session_id='f350dbbd-ab99-46dd-86a1-65d54ecdfcd1' data=FollowUp(status='follow_up_required', question_to_user='Do you have any other resources like an amplifier, picks, or online learning subscriptions that you can use for learning the guitar?')
i own a classical g

In [25]:
user_goal=DefinitionsCreate(title='Learn the guitar', metric="Passing the guitar club's interview", purpose='Joining the NUS guitar club', deadline=date(2026, 11, 14), status='definitions_extracted')
prereq=GoalPrerequisites(status='prerequisites_extracted', current_state=CurrentState(skill_level='Some experience', related_experience='Can play fingerstyle', resources_available='Classical guitar', user_gap_assessment=[], possible_gap_assessment=[]), fixed_resources=FixedResources(time_commitment_per_week_hours=0.5, budget=0.0, required_equipment='Classical guitar, Picks (not needed as user plays fingerstyle)', support_system='Friends'), constraints=Constraints(blocked_time_blocks=[], available_time_blocks=['Saturday afternoons', 'Sunday mornings'], dependencies=[]))
 
# this should be uploaded and pulled from the db instead
final = Definitions(id=1, title=user_goal.title, metric=user_goal.metric, purpose=user_goal.purpose, deadline=user_goal.deadline, prerequisites=prereq)
print(final)

title='Learn the guitar' metric="Passing the guitar club's interview" purpose='Joining the NUS guitar club' deadline=datetime.date(2026, 11, 14) status='definitions_extracted' id=1 prerequisites=GoalPrerequisites(status='prerequisites_extracted', current_state=CurrentState(skill_level='Some experience', related_experience='Can play fingerstyle', resources_available='Classical guitar', user_gap_assessment=[], possible_gap_assessment=[]), fixed_resources=FixedResources(time_commitment_per_week_hours=0.5, budget=0.0, required_equipment='Classical guitar, Picks (not needed as user plays fingerstyle)', support_system='Friends'), constraints=Constraints(blocked_time_blocks=[], available_time_blocks=['Saturday afternoons', 'Sunday mornings'], dependencies=[]))


In [33]:
PHASE_GENERATION_INSTRUCTION = """
You are an expert Strategic Planner. Your task is to analyze the user's defined goal and prerequisites to generate a realistic, sequential plan consisting of an appropriate amount of broad phases, depending on task complexity and total time. 
Each phase must have a measurable target and a clear start and end date.
**PhaseGeneration schema:** {phaseGeneration}
### Strict Generation Rules:
1.  **Output Format:** You **MUST** output a single JSON object that strictly conforms to the `PhaseGeneration` schema.
2.  **Phase Count:** Generate a list of sequential phases.
3.  **Measurability:** The `description` for each phase must contain a **tangible, measurable target** that acts as a stepping stone toward the final goal's `metric`.
4.  **Date Calculation:**
    * Use the user's `time_commitment_per_week_hours` to calculate a realistic duration for each phase.
    * The phases must be scheduled sequentially, starting from the current date ([CURRENT DATE: YYYY-MM-DD]), and leading up to the final goal's `deadline`.
    * The `end_date` of the final phase must be on or slightly before the overall goal's `deadline`.
    * Ensure the `start_date` of Phase N+1 immediately follows the `end_date` of Phase N.
5.  **Skill Integration:** The first phase should specifically address the user's current `skill_level` and any initial **gaps** identified in the `user_gap_assessment` or `possible_gap_assessment`.

The date is currently {current_date_str}.
The user's goals are defined as {goal}.
"""

client = genai.Client(api_key=GOOGLE_API_KEY)
response = client.models.generate_content(
    model='gemini-2.5-flash-lite',
    config={
        "system_instruction": PHASE_GENERATION_INSTRUCTION.format(
            current_date_str=date.today().strftime('%Y-%m-%d'),
            goal=final,
            phaseGeneration=PhaseGeneration.model_json_schema()
        ),
        "response_mime_type": "application/json",
        "response_schema": PhaseGeneration,
    },
    contents=["What should the plan look like?"]
)
print(response.parsed)

status='phases_generated' phases=[PhaseCreate(title='Repertoire Expansion and Technique Refinement', description='Learn and master at least 5 new songs across different genres (classical, folk, blues) suitable for performance, and solidify advanced fingerstyle techniques such as muting, hammer-ons, pull-offs, and string bending. Target: Perform these 5 songs smoothly without mistakes in front of friends.', start_date=datetime.date(2025, 11, 15), end_date=datetime.date(2026, 2, 14)), PhaseCreate(title='Music Theory and Ear Training for Guitarists', description='Develop a solid understanding of music theory relevant to guitar, including scales, chords, progressions, and modes. Improve ear training to identify intervals, chords, and melodies by ear. Target: Be able to identify and play by ear 10 common chord progressions and melodies from popular songs.', start_date=datetime.date(2026, 2, 15), end_date=datetime.date(2026, 5, 14)), PhaseCreate(title='Performance Preparation and Interview S

In [37]:
PHASE_REINEMENT_INSTRUCTION = """
You are an expert Strategic Plan Refiner. Your primary task is to maintain and revise the goal phase plan. Your single, non-negotiable output is the complete, revised plan in the PhaseGeneration schema.
**PhaseGeneration schema:** {phaseGeneration}

### Strict Refinement Rules:
1.  **Input Analysis:** Review the user's latest message (textual feedback on the plan) and integrate those changes into the existing plan structure found in your previous response(s).
2.  **Output:** You **MUST** output a single JSON object that strictly conforms to the `PhaseGeneration` schema. **The output must always be the full, complete, and revised plan.**
3.  **Plan Integrity:** If one phase's duration is changed, adjust the start_date and end_date of all subsequent phases accordingly to maintain sequential continuity and stay within the overall goal deadline.
4.  **No Questions/Chat:** You **MUST NOT** ask questions or provide conversational text. Your sole function is to output the revised JSON plan.

The date is currently {current_date_str}.
The user's goals are defined as {goal}.
"""
def phase_refinement(usr_reqeust = GoalRequest):
    if usr_reqeust.session_id:
        session_id = usr_reqeust.session_id
        chat = chat_sessions[session_id]
    else:
        session_id = str(uuid.uuid4())
        chat = client.chats.create(
            model='gemini-2.5-flash-lite',
            config={
                "system_instruction": PHASE_REINEMENT_INSTRUCTION.format(
                    current_date_str=date.today().strftime('%Y-%m-%d'),
                    goal=final,
                    phaseGeneration=PhaseGeneration.model_json_schema()
                ),
                "response_mime_type": "application/json",
                "response_schema": PhaseGeneration,
            },
        )
        chat_sessions[session_id] = chat

    response = chat.send_message(usr_reqeust.user_input)
    results = response.parsed
    return APIResponse(session_id=session_id, data=results)

import datetime
initial = PhaseGeneration(status='phases_generated', phases=[PhaseCreate(title='Repertoire Expansion and Technique Refinement', description='Learn and master at least 5 new songs across different genres (classical, folk, blues) suitable for performance, and solidify advanced fingerstyle techniques such as muting, hammer-ons, pull-offs, and string bending. Target: Perform these 5 songs smoothly without mistakes in front of friends.', start_date=datetime.date(2025, 11, 15), end_date=datetime.date(2026, 2, 14)), PhaseCreate(title='Music Theory and Ear Training for Guitarists', description='Develop a solid understanding of music theory relevant to guitar, including scales, chords, progressions, and modes. Improve ear training to identify intervals, chords, and melodies by ear. Target: Be able to identify and play by ear 10 common chord progressions and melodies from popular songs.', start_date=datetime.date(2026, 2, 15), end_date=datetime.date(2026, 5, 14)), PhaseCreate(title='Performance Preparation and Interview Simulation', description='Focus on polished performance of selected repertoire, stage presence, and improvisational skills. Simulate interview scenarios with guitar club members or friends, practicing answering questions about musical interests, influences, and performance experience. Target: Successfully perform a 10-minute set of chosen songs and confidently answer common interview questions.', start_date=datetime.date(2026, 5, 15), end_date=datetime.date(2026, 8, 14)), PhaseCreate(title='Final Polish and Interview Readiness', description="Incorporate feedback from simulations, refine performance pieces, and build confidence for the actual interview. Practice sight-reading simple musical passages if required by the club. Target: Achieve a level of performance confidence and technical proficiency that meets the assumed standards for the guitar club's interview.", start_date=datetime.date(2026, 8, 15), end_date=datetime.date(2026, 11, 14))])
for p in initial.phases:
    print(p)
usr_input = input("Any comments on the following plan?")
usr_parsed = f"With the current plan of {initial.phases}, I think that {usr_input}"
apiresponse = phase_refinement(GoalRequest(user_input=usr_parsed, session_id=None))
print(apiresponse)
print("---------")
for p in apiresponse.data.phases:
    print(p)
while usr_input != "q":
    usr_input = input("Any comments on the following plan?")
    print(usr_input)
    usr_parsed = f"With the current plan of {apiresponse.data.phases}, I think that {usr_input}"
    usr_reqeust = GoalRequest(user_input=usr_parsed, session_id=apiresponse.session_id)
    apiresponse = phase_refinement(usr_reqeust)
    print("---------")
    for p in apiresponse.data.phases:
        print(p)

title='Repertoire Expansion and Technique Refinement' description='Learn and master at least 5 new songs across different genres (classical, folk, blues) suitable for performance, and solidify advanced fingerstyle techniques such as muting, hammer-ons, pull-offs, and string bending. Target: Perform these 5 songs smoothly without mistakes in front of friends.' start_date=datetime.date(2025, 11, 15) end_date=datetime.date(2026, 2, 14)
title='Music Theory and Ear Training for Guitarists' description='Develop a solid understanding of music theory relevant to guitar, including scales, chords, progressions, and modes. Improve ear training to identify intervals, chords, and melodies by ear. Target: Be able to identify and play by ear 10 common chord progressions and melodies from popular songs.' start_date=datetime.date(2026, 2, 15) end_date=datetime.date(2026, 5, 14)
title='Performance Preparation and Interview Simulation' description='Focus on polished performance of selected repertoire, st